In [1]:
from buildingmotif.dataclasses import Library, Template, Model
from buildingmotif import BuildingMOTIF
from rdflib import Namespace
# this suppresses some warnings from the semantic graph synthesis implementation
import pandas
pandas.set_option('future.no_silent_downcasting', True)

Create the BuildingMOTIF instance. Load in the latest Brick library so we can validate our model against Brick when we are done. Also load the `equipment_templates` library, which contains some generic templates we can use to build our model.

In [2]:
bm = BuildingMOTIF('sqlite://')
Library.load(ontology_graph="https://brickschema.org/schema/1.4.1/Brick.ttl")
equipment_templates = Library.load(directory="pointlabel_demo/equipment_templates")

/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))
2024-09-17 20:48:18,686 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/overlay/qudt (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-17 20:48:18,687 | root |  WARNING: An ontology could not resolve a dependency on https://brickschema.org/schema/Brick/ref (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-17 20:48:18,688 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/bacnet/2020 (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-17 20:48:18,689 | root |  WARNING: An ontology could not resolve a

shape https://brickschema.org/schema/Brick#PV_Array has dependency https://brickschema.org/schema/Brick#PV_Panel
shape https://w3id.org/rec#RealEstate has dependency https://w3id.org/rec#Architecture
shape https://w3id.org/rec#ExceptionEvent has dependency https://brickschema.org/schema/Brick#Point
shape https://brickschema.org/schema/Brick/ref#IFCReference has dependency https://brickschema.org/schema/Brick/ref#ifcProject
shape https://w3id.org/rec#ActuationEvent has dependency https://brickschema.org/schema/Brick#Point
shape https://w3id.org/rec#Premises has dependency https://w3id.org/rec#Architecture
shape https://w3id.org/rec#Apartment has dependency https://w3id.org/rec#Room
shape https://w3id.org/rec#FurnitureCollection has dependency https://w3id.org/rec#Furniture
shape https://w3id.org/rec#Lease has dependency https://w3id.org/rec#Agent
shape https://w3id.org/rec#Lease has dependency https://w3id.org/rec#Agent
shape https://brickschema.org/schema/Brick#Loop has dependency http

2024-09-17 20:48:21,124 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7640 in libraries []
2024-09-17 20:48:21,127 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7644 in libraries []
2024-09-17 20:48:21,130 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7638 in libraries []
2024-09-17 20:48:21,134 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7636 in libraries []
2024-09-17 20:48:21,136 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7642 in libraries []
2024-09-17 20:48:21,141 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7699 in libraries []
2024-09-17 20:48:21,144 | root |  WARNING: Warning: could not find dependee n443b404c33df41a892628b660ba3a197b7705 in libraries []
2024-09-17 20:48:21,147 | root |  WARNING: Warning: could not find dependee n443b40

Declare a namespace for our model and create the (mostly empty) Model instance in BuildingMOTIF

In [3]:
BLDG = Namespace("urn:bldg/")
model = Model.create("urn:bldg")

We will convert this CSV of point labels into a Brick model. We could also do this from a BACnet network scan

In [4]:
# print first 5 lines
with open('pointlabel_demo/pointlabels.csv', 'r') as f:
    print(''.join(f.readlines()[:5]))

label
:BMOTIF_01:FCU048_ChwVlvPos
:BMOTIF_01:FCU048_UnoccHtgSptFnl
:BMOTIF_01:FCU191_BO4_HighSpdFanOut
:BMOTIF_01:FCU194_UO12_ChwVlvOut



We will use BuildingMOTIF's [parser combinator](https://buildingmotif.readthedocs.io/en/latest/explanations/point-label-parsing.html) library to create a function which breaks each label into its parts, and assigns types to those parts.

In [5]:
from buildingmotif.namespaces import BRICK
from buildingmotif.label_parsing.combinators import abbreviations, sequence, string, constant, regex, many, maybe, COMMON_EQUIP_ABBREVIATIONS_BRICK
from buildingmotif.label_parsing.tokens import Delimiter, Identifier, Constant

In [6]:
# define abbreviation dictionaries. Use a provided one for Equipment
equip_abbreviations = abbreviations(COMMON_EQUIP_ABBREVIATIONS_BRICK)
# define our own for Points (specific to this building)
point_abbreviations = abbreviations({
    "ChwVlvPos": BRICK.Position_Sensor,
    "HwVlvPos": BRICK.Position_Sensor,
    "RoomTmp": BRICK.Air_Temperature_Sensor,
    "Room_RH": BRICK.Relative_Humidity_Sensor,
    "UnoccHtgSpt": BRICK.Unoccupied_Air_Temperature_Heating_Setpoint,
    "OccHtgSpt": BRICK.Occupied_Air_Temperature_Heating_Setpoint,
    "UnoccClgSpt": BRICK.Unoccupied_Air_Temperature_Cooling_Setpoint,
    "OccClgSpt": BRICK.Occupied_Air_Temperature_Cooling_Setpoint,
    "SaTmp": BRICK.Supply_Air_Temperature_Sensor,
    "OccCmd": BRICK.Occupancy_Command,
    "EffOcc": BRICK.Occupancy_Status,
})

In [7]:
# now we can define the naming convention
def naming_convention(target):
    return sequence(
        string(":", Delimiter),
        # regex until the underscore
        constant(Constant(BRICK.Building)),
        regex(r"[^_]+", Identifier),
        string("_", Delimiter),
        # number for AHU name
        constant(Constant(BRICK.Air_Handling_Unit)),
        regex(r"[0-9a-zA-Z]+", Identifier),
        string(":", Delimiter),
        # equipment types
        equip_abbreviations,
        # equipment ident
        regex(r"[0-9a-zA-Z]+", Identifier),
        string("_", Delimiter),
        maybe(
            sequence(regex(r"[A-Z]+[0-9]+", Identifier), string("_", Delimiter)),
        ),
        # point types
        point_abbreviations,
    )(target)

We will assume the parser works for now. See the documentation above for help on how to develop and debug these parsers. BuildingMOTIF also has support for (a) building these parsers in an interactive web-based GUI, and (b) building these parsers automatically with GenerativeAI.

We can now feed our CSV of point labels through several [Ingress operators](https://buildingmotif.readthedocs.io/en/latest/explanations/ingresses.html). These are BuildingMOTIF's way of constructing pipelines of transformations, and are convenient for importing data into a semantic metadata model.

In [8]:
from buildingmotif.ingresses import CSVIngress, NamingConventionIngress, SemanticGraphSynthesizerIngress

# pull labels from the CSV file (requires a column called "label")
source = CSVIngress("pointlabel_demo/pointlabels.csv")
# parse each label using our 'naming_convention' function
ing = NamingConventionIngress(source, naming_convention)

In [9]:
# we can examine the (successful) output of the parser
ing.records[0]

Record(rtype='token', fields={'label': ':BMOTIF_01:FCU048_ChwVlvPos', 'tokens': [{'identifier': 'BMOTIF', 'type': 'https://brickschema.org/schema/Brick#Building'}, {'identifier': '01', 'type': 'https://brickschema.org/schema/Brick#Air_Handling_Unit'}, {'identifier': '048', 'type': 'https://brickschema.org/schema/Brick#Fan_Coil_Unit'}, {'identifier': ':BMOTIF_01:FCU048_ChwVlvPos', 'type': 'https://brickschema.org/schema/Brick#Position_Sensor'}]})

This next line will use the templates inside `equipment_templates` to group the tokens from each parsed label and assign them to the parameters of a template. See [the documentation](https://buildingmotif.readthedocs.io/en/latest/explanations/templates.html) for details on BuildingMOTIF templates.

In [10]:
sgs = SemanticGraphSynthesizerIngress(ing, [equipment_templates])

In [11]:
# add the results to our model
model.add_graph(sgs.graph(BLDG))

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "supply_temp, chw_coil, hw_coil-hw_vlv, chw_coil-position, chw_coil-chw_vlv, hw_coil-position, unocc_clg_sp, hw_coil, zone, unocc_htg_sp, occ_clg_sp, occ_htg_sp, room" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "supply_temp, chw_coil, hw_coil-hw_vlv, chw_coil-position, chw_coil-chw_vlv, hw_coil-position, hw_coil, unocc_clg_sp, zone, unocc_htg_sp, occ_clg_sp, occ_htg_sp, room" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "supply_temp, chw_coil, occ_cmd, hw_coil-hw_vlv, occ_status, chw_coil-chw_vlv, hw_coil-position, unocc_clg_sp, hw_coil, chw_coil-position, zone, unocc_htg_sp, occ_clg_sp, occ_htg_sp, room" were not provided during evaluation
  warnings.warn(


In [12]:
# print the resulting model (first 1000 chars)
print(model.graph.serialize()[:1000])

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg> a owl:Ontology .

<urn:building/048> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:bldg/zone_e4fe8e19> ;
    brick:hasPart <urn:bldg/chw_coil_0800ffa6>,
        <urn:bldg/hw_coil_7d883095> ;
    brick:hasPoint <urn:bldg/supply_temp_f975b5b9>,
        <urn:building/UI22> .

<urn:building/050> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:bldg/zone_9e8d3235> ;
    brick:hasPart <urn:bldg/chw_coil_d2beba75>,
        <urn:bldg/hw_coil_8b87336a> ;
    brick:hasPoint <urn:bldg/supply_temp_77e5b03a>,
        <urn:building/:BMOTIF_01:FCU050_EffOcc>,
        <urn:building/:BMOTIF_01:FCU050_OccCmd>,
        <urn:building/UI22> .

<urn:building/051> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:bldg/zone_e3584771> ;
    brick:hasPart <urn:bldg/chw_coil_78dabfa7>,
        <urn:bldg/hw_coil_112f13ba> ;
    brick:hasPoint <urn:bldg/supply_temp_e1e073d6>,
        <urn:building/UI22> .

<u